In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from tqdm import tqdm
import warnings
from datetime import datetime, timedelta
import statsmodels.api as sm
import os
warnings.filterwarnings("ignore")
os.getcwd()

'/Users/test/risksoft/TS/db_py'

In [2]:
try:
    os.chdir("../test123")
except FileNotFoundError:
    os.makedirs("../test123")
    os.chdir("../test123")

In [182]:
directory = 'tw/price'
list_ = os.listdir(directory)
df_list = [pd.read_parquet(os.path.join(directory, i)) for i in list_]
df_all = pd.concat(df_list)
df_all['da1'] = df_all['da'].apply(lambda x: f"{int(x[:3])+1911}-{x[4:6]}-{x[7:9]}")
df = df_all.drop_duplicates(subset=['ticker', 'da1'])
df['da_check'] = df['da1'].astype(str).apply(lambda x: int(x[:4]))
df_final = df[df['da_check'].isin([i for i in range(2018, 2026)])]

for values in ['cl', 'op', 'vol(volume)', 'vol(turnover)']:
    df_pivot = df_final.pivot(values=values, columns='ticker', index='da1')
    df_pivot.replace("--", None, inplace=True)
    df_pivot.replace("-", None, inplace=True)
    df_pivot.ffill(inplace=True)
    df_pivot = df_pivot.loc['2018-01-01':(datetime.now()+timedelta(days=1)).strftime("%Y-%m-%d")]
    df_pivot.sort_index(ascending=True, inplace=True)
    df_pivot = df_pivot.apply(lambda x: x.str.replace(',', '').astype(float))
    df_pivot.dropna(how='all', axis=1, inplace=True)
    df_pivot.to_parquet(f"tw/pdata/{values}.parquet")
    print(f"store pdata of values {values} to tw/pdata/{values}.parquet successful")
    df_pivot_pct = df_pivot.pct_change().dropna()
    df_pivot.to_parquet(f"tw/pdata/{values}_pct.parquet")
    print(f"store pdata of values {values} to tw/pdata/{values}_pct.parquet successful")
    


store pdata of values cl to tw/pdata/cl.parquet successful
store pdata of values cl to tw/pdata/cl_pct.parquet successful
store pdata of values op to tw/pdata/op.parquet successful
store pdata of values op to tw/pdata/op_pct.parquet successful
store pdata of values vol(volume) to tw/pdata/vol(volume).parquet successful
store pdata of values vol(volume) to tw/pdata/vol(volume)_pct.parquet successful
store pdata of values vol(turnover) to tw/pdata/vol(turnover).parquet successful
store pdata of values vol(turnover) to tw/pdata/vol(turnover)_pct.parquet successful


In [127]:
directory = 'tw/pb_ratio'
list_ = os.listdir(directory)
df_list = [pd.read_parquet(os.path.join(directory, i)) for i in list_]
df_all = pd.concat(df_list)
df_all['da'] = df_all.index
df = df_all.drop_duplicates(subset=['ticker', 'da'])
for values in ['pe', 'pb']:
    df_pivot = df.pivot(values=values, columns='ticker', index='da')

    df_pivot.replace("--", None, inplace=True)
    df_pivot.replace("-", None, inplace=True)
    df_pivot.ffill(inplace=True)
    df_pivot = df_pivot.loc['2018-01-01':(datetime.now()+timedelta(days=1)).strftime("%Y-%m-%d")]
    df_pivot.sort_index(ascending=True, inplace=True)
    df_pivot = df_pivot.apply(lambda x: x.str.replace(',', '').astype(float))
    df_pivot.dropna(how='all', axis=1, inplace=True)
    df_pivot.to_parquet(f"tw/pdata/{values}.parquet")
    print(f"store pdata of values {values} to tw/pdata/{values}.parquet successful")


store pdata of values pe to tw/pdata/pe.parquet successful
store pdata of values pb to tw/pdata/pb.parquet successful


### 富櫃50 TPEX

In [ ]:

import requests
list_list = []
for year in range(113, 102, -1):
    limit_month = 8 if year == 113 else 13
    for index, month in enumerate(range(1, limit_month)):
        month = f"0{month}" if month < 10 else month
        url_gretai50 = f"https://www.tpex.org.tw/web/stock/iNdex_info/gretai50/inxhis/rihisqry_result.php?l=zh-tw&d={year}/{month}/01&_=1720688879318"
        response = requests.get(url_gretai50)
        data = response.json()['aaData']
        list_list.append(data)
list_ = sum(list_list, [])

In [32]:
df = pd.DataFrame(list_)
df.to_parquet('../db/tw/ind/gretai50.parquet')

### 財報, selenium

In [35]:
from Selenium import Selenium

obj = Selenium()
obj.chromedriver_path



chromedriver already existed, for version 126.0


'/Users/test/risksoft/TS/db/chromedriver_mac/chromedriver-mac-x64/chromedriver'

In [2]:
from Selenium import Selenium
obj = Selenium()
obj.chromedriver_path

chromedriver already existed, for version 126.0


'/Users/test/risksoft/TS/db/chromedriver_mac/chromedriver-mac-x64/chromedriver'

In [ ]:
'''
cd /Users/test/risksoft/TS/db/chromedriver_mac/chromedriver-mac-x64/
chmod +x chromedriver

or

chmod +x /Users/test/risksoft/TS/db/chromedriver_mac/chromedriver-mac-x64/chromedriver

so that auto generated chromedriver can be reached and executed
'''

In [7]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 初始化 Selenium WebDriver
# options = webdriver.ChromeOptions()
# options.add_argument("--headless")
service = Service(executable_path=obj.chromedriver_path)

In [18]:
from selenium.webdriver.support.ui import Select
driver = webdriver.Chrome()
url = 'https://mops.twse.com.tw/mops/web/t164sb03'
driver.get(url)

wait = WebDriverWait(driver, 20)

select_element = wait.until(EC.presence_of_element_located((By.ID, 'isnew')))
select = Select(select_element)
select.select_by_visible_text('歷史資料')

company_code_input = wait.until(EC.presence_of_element_located((By.ID, 'co_id')))
company_code_input.send_keys('2330')

year_input = driver.find_element(By.ID, 'year')
year_input.send_keys('112')

select_element = wait.until(EC.presence_of_element_located((By.ID, 'season')))
select = Select(select_element)
select.select_by_visible_text('1')

search_button = driver.find_element(By.CSS_SELECTOR, 'input[type="button"][value=" 查詢 "]')
search_button.click()

# wait.until(EC.presence_of_element_located((By.ID, 'table01')))

# table = driver.find_element(By.ID, 'table01')
# rows = table.find_elements(By.TAG_NAME, 'tr')

# data = []
# for row in rows:
#     cols = row.find_elements(By.TAG_NAME, 'td')
#     cols = [col.text for col in cols]
#     data.append(cols)

# df = pd.DataFrame(data)

# print(df)


In [22]:
table = driver.find_element(By.CLASS_NAME, 'hasBorder')
rows = table.find_elements(By.TAG_NAME, "tr")
for row in rows:
    cols = row.find_elements(By.TAG_NAME, 'td')
    cols = [col.text for col in cols]
    data.append(cols)

In [24]:
data

[[],
 [],
 [],
 [],
 ['流動資產', '', '', '', '', '', ''],
 ['現金及約當現金',
  '1,385,232,810',
  '27.45',
  '1,342,814,083',
  '27.05',
  '1,151,589,646',
  '28.84'],
 ['透過損益按公允價值衡量之金融資產－流動',
  '982,014',
  '0.02',
  '1,070,398',
  '0.02',
  '758,452',
  '0.02'],
 ['透過其他綜合損益按公允價值衡量之金融資產－流動',
  '129,541,034',
  '2.57',
  '122,998,543',
  '2.48',
  '120,995,765',
  '3.03'],
 ['按攤銷後成本衡量之金融資產－流動',
  '73,428,595',
  '1.46',
  '94,600,219',
  '1.91',
  '8,425,386',
  '0.21'],
 ['避險之金融資產－流動', '70', '0.00', '2,329', '0.00', '286,824', '0.01'],
 ['應收帳款淨額',
  '147,715,125',
  '2.93',
  '229,755,887',
  '4.63',
  '212,846,891',
  '5.33'],
 ['應收帳款－關係人淨額', '332,554', '0.01', '1,583,958', '0.03', '581,010', '0.01'],
 ['其他應收款－關係人淨額', '76,608', '0.00', '68,975', '0.00', '58,165', '0.00'],
 ['存貨', '216,067,981', '4.28', '221,149,148', '4.45', '200,118,682', '5.01'],
 ['其他流動資產', '42,350,730', '0.84', '38,853,204', '0.78', '26,576,811', '0.67'],
 ['流動資產合計',
  '1,995,727,521',
  '39.55',
  '2,052,896,744',
  '41.

In [25]:
pd.DataFrame(data[4:])

,0,1,2,3,4,5,6
0,流動資產,,,,,,
1,現金及約當現金,"1,385,232,810",27.45,"1,342,814,083",27.05,"1,151,589,646",28.84
2,透過損益按公允價值衡量之金融資產－流動,"982,014",0.02,"1,070,398",0.02,"758,452",0.02
3,透過其他綜合損益按公允價值衡量之金融資產－流動,"129,541,034",2.57,"122,998,543",2.48,"120,995,765",3.03
4,按攤銷後成本衡量之金融資產－流動,"73,428,595",1.46,"94,600,219",1.91,"8,425,386",0.21
...,...,...,...,...,...,...,...
65,權益總額,"3,092,897,598",61.30,"2,960,488,867",59.63,"2,321,468,754",58.14
66,負債及權益總計,"5,045,844,348",100.00,"4,964,778,878",100.00,"3,992,676,667",100.00
67,待註銷股本股數（單位：股）,"419,466",,0,,0,
68,預收股款（權益項下）之約當發行股數（單位：股）,0,,0,,0,
